In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [106]:
data = pd.read_csv("bank.csv")

In [107]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [108]:
#Surname is also dropped as it is test data provides no impact on assessing customer retention probability
data.drop(columns = ['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


Target set would be the 'isActive' column since it checks whether bank customer is active, which would be useful for
validating the possibility of a bank customer staying or leaving.

In [ ]:
X = data.loc[:, data.columns != 'IsActiveMember']
Y = data.loc[:, data.columns == 'IsActiveMember']

In [110]:
data[data.isnull().any(axis=1)]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


Observation is '0' null values in both training and test set

In [111]:
data.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

In [112]:
#Checking Categorical Values in X
print(data['Geography'].unique())
print(data['Gender'].unique())

['France' 'Spain' 'Germany']
['Female' 'Male']


In [113]:
#Converting Categorical Values to Numeric Format, dropping redundant first dummy variable for decorrelation
data = pd.get_dummies(data, columns = ['Geography', 'Gender'], dtype = np.int64, drop_first = True)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null int64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Geography_Germany    10000 non-null int64
Geography_Spain      10000 non-null int64
Gender_Male          10000 non-null int64
dtypes: float64(2), int64(10)
memory usage: 937.6 KB


In [117]:
#Capping Outliers in the given dataset. We shall only consider features which shall qualify for outlier analysis i.e.
#features with continuous values and not features that are categorical, order or count variables as they are ineligible for outlier
#analysis.

capDf = pd.DataFrame()
toCapDf = data.loc[:, ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']]
for col in data.columns:
    if(data[col].dtype == object):
        capDf[col] = data[col]
        continue
    if(col in toCapDf.columns):
        percentiles = toCapDf[col].quantile([0.25,0.75]).values
        Q1 = percentiles[0]
        Q3 = percentiles[1]
        IQR = Q3 - Q1
        minCap = Q1 - (IQR) * 1.5
        maxCap = Q3 + (IQR) * 1.5
        capDf[col] = toCapDf[col][(toCapDf[col] >= minCap) & (toCapDf[col] <= maxCap)]
    else:
        capDf[col] = data[col]
    
capDf.dropna(inplace=True);

In [118]:
#Printing the outliers which were removed (In all, 374 eligible outlier rows were removed)
outlierDf = pd.concat([data, capDf])
outlierDf = outlierDf.reset_index(drop=True)
df_gpby = outlierDf.groupby(list(outlierDf.columns))
idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
outlierDf.reindex(idx)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
1838,350,39.0,0,109733.20,2,0,0,123602.11,1,1,0,1
9624,350,40.0,0,111098.85,1,1,1,172321.21,1,0,0,0
8723,350,51.0,10,0.00,1,1,1,125823.79,1,0,0,1
1631,350,54.0,1,152677.48,1,1,1,191973.49,1,0,1,1
8762,350,60.0,3,0.00,1,0,0,113796.15,1,0,0,0
2473,351,57.0,4,163146.46,1,1,0,169621.69,1,1,0,0
1962,358,52.0,8,143542.36,3,1,0,141959.11,1,0,1,0
1405,359,44.0,6,128747.69,1,1,0,146955.71,1,0,0,0
1193,363,28.0,6,146098.43,3,1,0,100615.14,1,0,1,0
2579,365,30.0,0,127760.07,1,1,0,81537.85,1,1,0,1


Target set would be the 'isActive' column since it checks whether bank customer is active, which would be useful for
validating the possibility of a bank customer staying or leaving.

In [124]:
capDf.info()
X = capDf.loc[:, capDf.columns != 'IsActiveMember']
Y = capDf.loc[:, capDf.columns == 'IsActiveMember']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9626 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          9626 non-null int64
Age                  9626 non-null float64
Tenure               9626 non-null int64
Balance              9626 non-null float64
NumOfProducts        9626 non-null int64
HasCrCard            9626 non-null int64
IsActiveMember       9626 non-null int64
EstimatedSalary      9626 non-null float64
Exited               9626 non-null int64
Geography_Germany    9626 non-null int64
Geography_Spain      9626 non-null int64
Gender_Male          9626 non-null int64
dtypes: float64(3), int64(9)
memory usage: 977.6 KB


In [143]:
print(X.info())
print(Y.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9626 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore          9626 non-null int64
Age                  9626 non-null float64
Tenure               9626 non-null int64
Balance              9626 non-null float64
NumOfProducts        9626 non-null int64
HasCrCard            9626 non-null int64
EstimatedSalary      9626 non-null float64
Exited               9626 non-null int64
Geography_Germany    9626 non-null int64
Geography_Spain      9626 non-null int64
Gender_Male          9626 non-null int64
dtypes: float64(3), int64(8)
memory usage: 902.4 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9626 entries, 0 to 9999
Data columns (total 1 columns):
IsActiveMember    9626 non-null int64
dtypes: int64(1)
memory usage: 150.4 KB
None


In [188]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [189]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [190]:
X_train_scaled.head(1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,0.949542,-1.446948,-0.355142,0.890415,0.814713,0.646244,0.523978,-0.504032,-0.583863,-0.573352,-1.091757


In [191]:
X_test_scaled.head(1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,0.158655,-0.084361,-1.046646,1.31239,-0.923582,-1.547404,-0.109943,-0.504032,1.712732,-0.573352,-1.091757


In [194]:
# baseline model
def create_baseline():
    # create model
    model = keras.Sequential()
    model.add(keras.layers.Dense(30, input_dim = 11, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [195]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
#Training Dataset
estimator = keras.wrappers.scikit_learn.KerasClassifier(build_fn = create_baseline, epochs = 50, batch_size = 5, verbose=0)
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
results = cross_val_score(estimator, np.array(X_train_scaled), np.array(y_train), cv = kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 53.58% (1.14%)


## Separate Approach

In [205]:
#Initialize Sequential model
model_new = tf.keras.models.Sequential()

#Normalize the data
model_new.add(tf.keras.layers.BatchNormalization())

model_new.add(keras.layers.Dense(30, input_dim = 11, kernel_initializer='normal', activation='relu'))

model_new.add(keras.layers.Dense(1, input_dim = 11, kernel_initializer='normal', activation='relu'))

#Add Dense Layer which provides 2 Outputs after applying sigmoid
model_new.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#Comile the model
model_new.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [206]:
model_new.fit(np.array(X_train_scaled), np.array(y_train), 
          validation_data=(np.array(X_test_scaled), np.array(y_test)), 
          epochs = 50,
          batch_size=X_train_scaled.shape[0])

Train on 6738 samples, validate on 2888 samples
Epoch 1/50
6738/6738 [==============================] - 1s 79us/sample - loss: 0.6930 - acc: 0.5190 - val_loss: 0.6928 - val_acc: 0.5062
Epoch 2/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6929 - acc: 0.5024 - val_loss: 0.6928 - val_acc: 0.5062
Epoch 3/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6928 - acc: 0.5024 - val_loss: 0.6927 - val_acc: 0.5062
Epoch 4/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6927 - acc: 0.5024 - val_loss: 0.6926 - val_acc: 0.5062
Epoch 5/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6926 - acc: 0.5024 - val_loss: 0.6925 - val_acc: 0.5062
Epoch 6/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6925 - acc: 0.5024 - val_loss: 0.6924 - val_acc: 0.5062
Epoch 7/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6924 - acc: 0.5024 - val_loss: 0.6923 - val_acc: 0.5329

Thus, Old and New Models provide similar accuracy with the highest being 53.6% using crossvalidation on the Keras Scikit Learn Classification Wrapper and 55.6% manually using the adam optimizer.

In [209]:
from sklearn.model_selection import GridSearchCV

#Let us try with Grid Search, with different learning rates on SGD
#Initialize Sequential model
model_new_addtl = tf.keras.models.Sequential()

#Normalize the data
model_new_addtl.add(tf.keras.layers.BatchNormalization())

model_new_addtl.add(keras.layers.Dense(30, input_dim = 11, kernel_initializer='normal', activation='relu'))

model_new_addtl.add(keras.layers.Dense(1, input_dim = 11, kernel_initializer='normal', activation='relu'))

#Add Dense Layer which provides 2 Outputs after applying sigmoid
model_new_addtl.add(tf.keras.layers.Dense(1, activation='sigmoid'))

sgd = keras.optimizers.SGD(lr = 0.01)

#Comile the model
model_new_addtl.compile(optimizer = sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [210]:
model_new_addtl.fit(np.array(X_train_scaled), np.array(y_train), 
          validation_data=(np.array(X_test_scaled), np.array(y_test)), 
          epochs = 50,
          batch_size=X_train_scaled.shape[0])

Train on 6738 samples, validate on 2888 samples
Epoch 1/50
6738/6738 [==============================] - 0s 71us/sample - loss: 0.6931 - acc: 0.4981 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 2/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 3/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 4/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 5/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 6/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 7/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062

In [204]:
#Fall in accuracy using sigmoid

In [211]:
#Increasing the Learning Rate
sgd = keras.optimizers.SGD(lr = 0.1)

#Comile the model
model_new_addtl.compile(optimizer = sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [212]:
model_new_addtl.fit(np.array(X_train_scaled), np.array(y_train), 
          validation_data=(np.array(X_test_scaled), np.array(y_test)), 
          epochs = 50,
          batch_size=X_train_scaled.shape[0])

Train on 6738 samples, validate on 2888 samples
Epoch 1/50
6738/6738 [==============================] - 0s 73us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 2/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 3/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 4/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 5/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 6/50
6738/6738 [==============================] - 0s 2us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 7/50
6738/6738 [==============================] - 0s 1us/sample - loss: 0.6931 - acc: 0.5024 - val_loss: 0.6931 - val_acc: 0.5062

In [213]:
#No Change

## Conclusion

Manual Model Fitting with Adam Optimizer and Batch Normalization provided the best results with a highest accuracy on test set of 55.6% while using Stratified K Fold Cross Validation proved computationally expensive at the same time compromising on the results.